In [2]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from fuzzywuzzy import fuzz

C:\Users\Hussein\AppData\Local\Temp\ipykernel_11852\4204607573.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Load and Test Pretrained Inception Model 

In [3]:
# Load the pre-trained InceptionV3 model
model = InceptionV3(weights='imagenet')

# Function to preprocess the image and predict its label
def predict_image_label(img_path):
    # Load the image file, resizing it to 299x299 pixels (as required by InceptionV3)
    img = image.load_img(img_path, target_size=(299, 299))
    
    # Convert the image to a numpy array and add an additional dimension (for batch size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess the image for the InceptionV3 model
    img_array = preprocess_input(img_array)
    
    # Predict the probabilities across all output classes
    predictions = model.predict(img_array)
    
    # Decode the predictions to get human-readable labels
    decoded_predictions = decode_predictions(predictions, top=3)[0]
    predictions_list = []
    print(f"Predicted labels for {os.path.basename(img_path)} (top-3):")
    for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
        print(f"{i+1}: {label} ({score*100:.2f}%)")
        predictions_list.append((label, score))
    return predictions_list

# Path to the folder containing images
folder_path = 'Example Data-20240208T214429Z-001/Example Data/exported'

predictions_dict = {}
# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):  # Check for common image file extensions
        img_path = os.path.join(folder_path, filename)
        predictions_dict[filename] = predict_image_label(img_path)

# Print the filenames and corresponding predicted labels
for filename, predicted_labels in predictions_dict.items():
    print(f"\nFilename: {filename}")
    print("Predicted labels (top-3):")
    for i, (label, score) in enumerate(predicted_labels):
        print(f"{i+1}: {label} ({score*100:.2f}%)")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted labels for (1) 18380579401063495.jpg (top-3):
1: motor_scooter (90.88%)
2: moped (1.75%)
3: snowmobile (1.61%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Predicted labels for (1) 18380579401063495.png (top-3):
1: motor_scooter (89.87%)
2: snowmobile (1.72%)
3: moped (1.67%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Predicted labels for (1) @GreyCupFestival - 109th Grey Cup.jpeg (top-3):
1: stage (58.90%)
2: moving_van (2.70%)
3: mortarboard (1.68%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Predicted labels for (10) 17887803224903630.jpeg (top-3):
1: seashore (39.63%)
2: sandbar (7.99%)
3: Eskimo_dog (3.33%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Predicted labels for (11) 17997439897932301.png (top-3):
1: crash_helmet (60.60%)
2: motor_scooter (5.71%)
3: moped (3.50%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Predicted labels for (12) 17985809330117499.jpeg (top-3):
1: moped (63.87%)
2: motor_scooter (16.93%)
3: crash_helmet (8.56%)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0

Preprocces the images in the images folder

In [4]:
def preprocess_image(img_path):
    try:
        # Load the image file, resizing it to 299x299 pixels (as required by InceptionV3)
        img = image.load_img(img_path, target_size=(299, 299))
        
        # Convert the image to a numpy array
        img_array = image.img_to_array(img)
        
        # Add a dimension to the array for batch size
        img_array = np.expand_dims(img_array, axis=0)
        
        # Apply the specific preprocessing required by InceptionV3
        img_array = preprocess_input(img_array)
        
        return img_array
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None

In [5]:
def preprocess_images_in_folder(folder_path):
    preprocessed_images = []
    
    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):  # Check for common image file extensions
            img_path = os.path.join(folder_path, filename)
            img_array = preprocess_image(img_path)
            if img_array is not None:
                preprocessed_images.append(img_array)
    
    return preprocessed_images

# Preprocess all images in the specified folder
preprocessed_images = preprocess_images_in_folder(folder_path)

Extracting the image labels from excel file

In [6]:
excel_path = 'Example Data-20240208T214429Z-001/Example Data/Imagelabels.xlsx'


# Function to read the Excel file and extract image names and labels
def read_labels_from_excel(excel_path):
    # Read the Excel file
    df = pd.read_excel(excel_path)
    
    # the Excel file has columns 'Image Name' and 'Label'
    labels_dict = pd.Series(df.Label.values, index=df['Image Name']).to_dict()
    
    return labels_dict

# Call the function and store the result in a variable
actual_labels_dict = read_labels_from_excel(excel_path)


In [7]:
def preprocess_image(img_path):
    try:
        img = image.load_img(img_path, target_size=(299, 299))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        return img_array
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        return None

def predict_image_label(img_path):
    img_array = preprocess_image(img_path)
    if img_array is not None:
        predictions = model.predict(img_array)
        decoded_predictions = decode_predictions(predictions, top=1)[0]
        return decoded_predictions[0][1]  # Return only the top prediction label
    return None

def read_labels_from_excel(excel_path):
    df = pd.read_excel(excel_path)
    labels_dict = pd.Series(df.Label.values, index=df['Image Name']).to_dict()
    return labels_dict

# Function to compare predicted and actual labels for similarity
def compare_label_similarity(predicted_label, actual_label):
    return fuzz.ratio(predicted_label.lower(), actual_label.lower())

# Main processing function
def process_images_and_compare_labels(folder_path, excel_path):
    actual_labels_dict = read_labels_from_excel(excel_path)
    similarities = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder_path, filename)
            predicted_label = predict_image_label(img_path)
            actual_label = actual_labels_dict.get(filename)

            if predicted_label and actual_label:
                similarity_score = compare_label_similarity(predicted_label, actual_label)
                similarities.append(similarity_score)
                print(f"{filename}: Predicted Label - {predicted_label}, Actual Label - {actual_label}, Similarity - {similarity_score}%")

    if similarities:
        average_similarity = sum(similarities) / len(similarities)
        print(f"Average Label Similarity: {average_similarity}%")
    else:
        print("No images processed.")

# Update with your actual folder and Excel paths
folder_path = 'Example Data-20240208T214429Z-001/Example Data/exported'
excel_path = 'Example Data-20240208T214429Z-001/Example Data/Imagelabels.xlsx'

process_images_and_compare_labels(folder_path, excel_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
(1) @GreyCupFestival - 109th Grey Cup.jpeg: Predicted Label - stage, Actual Label - Glasses, Similarity - 33%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
(10) 17887803224903630.jpeg: Predicted Label - seashore, Actual Label - Woman, Similarity - 15%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
(12) 17985809330117499.jpeg: Predicted Label - moped, Actual Label - Night, Similarity - 0%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
(13) 18013990822817757.jpeg: Predicted Label - jersey, Actual Label - Paper, Similarity - 36%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
(14) 17993584322154200.jpeg: Predicted Label - cliff, Actual Label - Lion, Similarity - 44%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
(15) 18346855723078911.jpeg: Predicted Label - street_sign, Actual Label - Poster, Similarity - 35%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
(16) 18379894042056715.jpeg: Predict

In [8]:

# Main processing and evaluation function
def evaluate_image_labeling_system(folder_path, excel_path, similarity_threshold=80):
    actual_labels_dict = read_labels_from_excel(excel_path)
    similarities = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder_path, filename)
            predicted_label = predict_image_label(img_path)
            actual_label = actual_labels_dict.get(filename)

            if predicted_label and actual_label:
                similarity_score = compare_label_similarity(predicted_label, actual_label)
                similarities.append(similarity_score)
                print(f"{filename}: Predicted - {predicted_label}, Actual - {actual_label}, Similarity - {similarity_score}%")

    if similarities:
        average_similarity = sum(similarities) / len(similarities)
        print(f"\nAverage Label Similarity: {average_similarity}%")
        
        if average_similarity >= similarity_threshold:
            print(f"The system meets the required similarity threshold of {similarity_threshold}%. System evaluation passed.")
        else:
            print(f"The system does not meet the required similarity threshold of {similarity_threshold}%. System evaluation failed.")
    else:
        print("No images processed. Evaluation cannot be performed.")


evaluate_image_labeling_system(folder_path, excel_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
(1) @GreyCupFestival - 109th Grey Cup.jpeg: Predicted - stage, Actual - Glasses, Similarity - 33%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
(10) 17887803224903630.jpeg: Predicted - seashore, Actual - Woman, Similarity - 15%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
(12) 17985809330117499.jpeg: Predicted - moped, Actual - Night, Similarity - 0%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
(13) 18013990822817757.jpeg: Predicted - jersey, Actual - Paper, Similarity - 36%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
(14) 17993584322154200.jpeg: Predicted - cliff, Actual - Lion, Similarity - 44%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
(15) 18346855723078911.jpeg: Predicted - street_sign, Actual - Poster, Similarity - 35%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
(16) 18379894042056715.jpeg: Predicted - cowboy_hat, Actual - Head, Similarity - 29%
1/1 ━━━━━━━━━━━━━━━━━━━

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

Fine tune the pretrained model to get better results

Step 1: Preparing the Dataset


In [12]:
# Load and preprocess labels
excel_path = 'Example Data-20240208T214429Z-001/Example Data/Imagelabels.xlsx'
df = pd.read_excel(excel_path)

# Assuming 'Image Name' column has the image filenames and 'Label' column has labels separated by some delimiter (e.g., ',')
df['Label'] = df['Label'].apply(lambda x: x.split(','))  # Split labels into lists
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(df['Label'])

# Assuming image filenames are unique and map one-to-one to labels
labels_dict = dict(zip(df['Image Name'], labels))

# Load and preprocess images
folder_path = 'Example Data-20240208T214429Z-001/Example Data/exported'
image_size = (299, 299)  # InceptionV3 default image size

def load_and_preprocess_image(path):
    img = image.load_img(path, target_size=image_size)
    img_array = image.img_to_array(img)
    return preprocess_input(img_array)

#  images are stored directly in folder_path with filenames matching those in the Excel file
images = [load_and_preprocess_image(os.path.join(folder_path, fname)) for fname in labels_dict.keys()]
images = np.array(images)

# Split data into training and validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images, list(labels_dict.values()), test_size=0.2, random_state=42)

Step 2: Modifying and Training the Model

In [15]:


# Load InceptionV3 pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add new layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(mlb.classes_), activation='softmax')(x)  # 'sigmoid' for multi-label classification

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the new data
model.fit(X_train, np.array(y_train), validation_data=(X_val, np.array(y_val)), epochs=100, batch_size=32)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.0000e+00 - loss: 0.7332 - val_accuracy: 0.0000e+00 - val_loss: 0.2945
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 0.3108 - val_accuracy: 0.0000e+00 - val_loss: 0.0959
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 0.1147 - val_accuracy: 0.0000e+00 - val_loss: 0.0390
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: 0.0499 - val_accuracy: 0.0000e+00 - val_loss: 0.0354
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0417 - loss: 0.0377 - val_accuracy: 0.0000e+00 - val_loss: 0.0415
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1667 - loss: 0.0358 - val_accuracy: 0.1429 - val_loss: 0.0479
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1667 - loss: 0.0336 - val_accuracy: 0.1429 - val_loss: 0.0532
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1667 - loss: 0.0298 - val_accuracy